### Author: Dr. Rami Al-Salman
### Email: alsalman.rami@gmail.com
### This notebook aims to walk you through ORM concepts and practically and programmtically you can connect and manuplicate database tables of pagila database. 

In [3]:
#pip sqlalchemy psycopg2

# Simple ORM example to represent Actor object and interact with it 

In [2]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

username = 'postgres'
password = 'root'
host = 'localhost'
database = 'pagila'
# Step 1: Define the connection string to your PostgreSQL database
DATABASE_URL = f"postgresql+psycopg2://{username}:{password}@{host}:5433/{database}"

# Step 2: Create an engine
engine = create_engine(DATABASE_URL)

# Step 3: Define a base class for declarative class definitions
Base = declarative_base()

# Step 4: Define the ORM model for the actor table
class Actor(Base):
    __tablename__ = 'actor'
    
    actor_id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String, nullable=False)
    last_name = Column(String, nullable=False)

    def __repr__(self):
        return f"<Actor(actor_id={self.actor_id}, first_name={self.first_name}, last_name={self.last_name})>"

# Step 5: Create the tables in the database (if not already created)
#Base.metadata.create_all(engine)

# Step 6: Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Step 7: Query the database (Example: Get all actors)
actors = session.query(Actor).all()
for actor in actors:
    print(actor)

# Step 8: Close the session
session.close()


<Actor(actor_id=1, first_name=PENELOPE, last_name=GUINESS)>
<Actor(actor_id=2, first_name=NICK, last_name=WAHLBERG)>
<Actor(actor_id=3, first_name=ED, last_name=CHASE)>
<Actor(actor_id=4, first_name=JENNIFER, last_name=DAVIS)>
<Actor(actor_id=5, first_name=JOHNNY, last_name=LOLLOBRIGIDA)>
<Actor(actor_id=6, first_name=BETTE, last_name=NICHOLSON)>
<Actor(actor_id=7, first_name=GRACE, last_name=MOSTEL)>
<Actor(actor_id=8, first_name=MATTHEW, last_name=JOHANSSON)>
<Actor(actor_id=9, first_name=JOE, last_name=SWANK)>
<Actor(actor_id=10, first_name=CHRISTIAN, last_name=GABLE)>
<Actor(actor_id=11, first_name=ZERO, last_name=CAGE)>
<Actor(actor_id=12, first_name=KARL, last_name=BERRY)>
<Actor(actor_id=13, first_name=UMA, last_name=WOOD)>
<Actor(actor_id=14, first_name=VIVIEN, last_name=BERGEN)>
<Actor(actor_id=15, first_name=CUBA, last_name=OLIVIER)>
<Actor(actor_id=16, first_name=FRED, last_name=COSTNER)>
<Actor(actor_id=17, first_name=HELEN, last_name=VOIGHT)>
<Actor(actor_id=18, first_name

/var/folders/c1/8mhcv8nx2b9dswvv8thjqxjc0000gn/T/ipykernel_17237/1475482560.py:16: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## More advanced example with different tables relationships

In [12]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

# Database connection string
username = 'postgres'
password = 'root'
host = 'localhost'
database = 'pagila'
# Step 1: Define the connection string to your PostgreSQL database
DATABASE_URL = f"postgresql+psycopg2://{username}:{password}@{host}:5433/{database}"

# Create an engine that connects to the PostgreSQL database
engine = create_engine(DATABASE_URL)

# Create a base class for ORM models to inherit from
Base = declarative_base()

# Define an association table for the many-to-many relationship between Film and Actor
film_actor = Table(
    'film_actor', Base.metadata,
    Column('actor_id', Integer, ForeignKey('actor.actor_id'), primary_key=True),
    Column('film_id', Integer, ForeignKey('film.film_id'), primary_key=True)
)

class Actor(Base):
    """
    ORM model for the 'actor' table. Represents an actor with first and last names.
    
    Attributes:
        actor_id (int): Primary key, unique identifier for the actor.
        first_name (str): The first name of the actor.
        last_name (str): The last name of the actor.
    """
    __tablename__ = 'actor'

    actor_id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String, nullable=False)
    last_name = Column(String, nullable=False)

    # Relationship to films through the association table
    films = relationship('Film', secondary=film_actor, back_populates='actors')

    def __repr__(self):
        """Provides a clear string representation of the Actor object."""
        return f"<Actor(actor_id={self.actor_id}, first_name={self.first_name}, last_name={self.last_name})>"

class Film(Base):
    """
    ORM model for the 'film' table. Represents a film with a title, description, release year, and language.
    
    Attributes:
        film_id (int): Primary key, unique identifier for the film.
        title (str): The title of the film.
        description (str): The description of the film.
        release_year (int): The release year of the film.
        language_id (int): Foreign key linking to the 'language' table.
    """
    __tablename__ = 'film'

    film_id = Column(Integer, primary_key=True, autoincrement=True)
    title = Column(String, nullable=False)
    description = Column(String)
    release_year = Column(Integer)
    language_id = Column(Integer, ForeignKey('language.language_id'), nullable=False)

    # Relationship to actors through the association table
    actors = relationship('Actor', secondary=film_actor, back_populates='films')

    def __repr__(self):
        """Provides a clear string representation of the Film object."""
        return f"<Film(film_id={self.film_id}, title={self.title})>"

class Category(Base):
    """
    ORM model for the 'category' table. Represents a category for films (e.g., Action, Comedy).
    
    Attributes:
        category_id (int): Primary key, unique identifier for the category.
        name (str): The name of the category.
    """
    __tablename__ = 'category'

    category_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False)

    # Relationship to films through the association table
    films = relationship('Film', secondary='film_category', back_populates='categories')

    def __repr__(self):
        """Provides a clear string representation of the Category object."""
        return f"<Category(category_id={self.category_id}, name={self.name})>"

class FilmCategory(Base):
    """
    Association table for the many-to-many relationship between films and categories.
    
    Attributes:
        film_id (int): Foreign key linking to the 'film' table.
        category_id (int): Foreign key linking to the 'category' table.
    """
    __tablename__ = 'film_category'

    film_id = Column(Integer, ForeignKey('film.film_id'), primary_key=True)
    category_id = Column(Integer, ForeignKey('category.category_id'), primary_key=True)



# Update the Film model to include the relationship with Category
Film.categories = relationship('Category', secondary='film_category', back_populates='films')

# Create all tables in the database (if they don't exist)
Base.metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# Example Query: Find all films for a specific actor
actor = session.query(Actor).filter_by(first_name='JULIA').first()
print(f"Films featuring {actor.first_name} {actor.last_name}:")
for film in actor.films:
    print(f"- {film.title}")

# Example Query: Find all actors for a specific film
film = session.query(Film).filter_by(title='ACADEMY DINOSAUR').first()
print(f"Actors in the film {film.title}:")
for actor in film.actors:
    print(f"- {actor.first_name} {actor.last_name}")

# Example Query: Find all films in a specific category
category = session.query(Category).filter_by(name='Action').first()
print(f"Films in the {category.name} category:")
for film in category.films:
    print(f"- {film.title}")


# Close the session after completing the queries
session.close()


Films featuring JULIA MCQUEEN:
- AMADEUS HOLY
- ARABIA DOGMA
- BONNIE HOLOCAUST
- CIDER DESIRE
- CONEHEADS SMOOCHY
- EFFECT GLADIATOR
- FREDDY STORM
- GAMES BOWFINGER
- GLADIATOR WESTWARD
- HANOVER GALAXY
- HIGH ENCINO
- INSIDER ARIZONA
- JAWBREAKER BROOKLYN
- KISS GLORY
- KRAMER CHOCOLATE
- LUCKY FLYING
- MOCKINGBIRD HOLLYWOOD
- MONTEREY LABYRINTH
- OPEN AFRICAN
- PILOT HOOSIERS
- PITTSBURGH HUNCHBACK
- PRESIDENT BANG
- SCORPION APOLLO
- SLEEPLESS MONSOON
- SPIRIT FLINTSTONES
- STRANGERS GRAFFITI
- SWEETHEARTS SUSPECTS
- TELEMARK HEARTBREAKERS
- TIES HUNGER
- TRAIN BUNCH
- WEEKEND PERSONAL
- WONKA SEA
- YOUNG LANGUAGE
Actors in the film ACADEMY DINOSAUR:
- PENELOPE GUINESS
- CHRISTIAN GABLE
- LUCILLE TRACY
- SANDRA PECK
- JOHNNY CAGE
- MENA TEMPLE
- WARREN NOLTE
- OPRAH KILMER
- ROCK DUKAKIS
- MARY KEITEL
Films in the Action category:
- AMADEUS HOLY
- AMERICAN CIRCUS
- ANTITRUST TOMATOES
- ARK RIDGEMONT
- BAREFOOT MANCHURIAN
- BERETS AGENT
- BRIDE INTRIGUE
- BULL SHAWSHANK
- CADDYSHAC

/var/folders/c1/8mhcv8nx2b9dswvv8thjqxjc0000gn/T/ipykernel_14973/3690622991.py:17: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## More advanced example with different tables relationships and applying insert and update operations

In [19]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

# Database connection string
username = 'postgres'
password = 'root'
host = 'localhost'
database = 'pagila'
# Step 1: Define the connection string to your PostgreSQL database
DATABASE_URL = f"postgresql+psycopg2://{username}:{password}@{host}:5433/{database}"

# Create an engine that connects to the PostgreSQL database
engine = create_engine(DATABASE_URL)

# Create a base class for ORM models to inherit from
Base = declarative_base()

# Association table for the many-to-many relationship between Film and Actor
film_actor = Table(
    'film_actor', Base.metadata,
    Column('actor_id', Integer, ForeignKey('actor.actor_id'), primary_key=True),
    Column('film_id', Integer, ForeignKey('film.film_id'), primary_key=True)
)

class Actor(Base):
    """
    ORM model representing an actor in the database.
    
    Relationships:
    - `films`: Many-to-many relationship with the `Film` model through the `film_actor` association table.

    Attributes:
        actor_id (int): Primary key, unique identifier for the actor.
        first_name (str): The first name of the actor.
        last_name (str): The last name of the actor.
    """
    __tablename__ = 'actor'

    actor_id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String, nullable=False)
    last_name = Column(String, nullable=False)

    # Many-to-many relationship with Film
    films = relationship('Film', secondary=film_actor, back_populates='actors')

    def __repr__(self):
        return f"<Actor(actor_id={self.actor_id}, first_name={self.first_name}, last_name={self.last_name})>"

class Language(Base):
    """
    ORM model representing a language in the database.
    
    Relationships:
    - `films`: One-to-many relationship with the `Film` model.

    Attributes:
        language_id (int): Primary key, unique identifier for the language.
        name (str): The name of the language (e.g., 'English').
    """
    __tablename__ = 'language'

    language_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False)

    # One-to-many relationship with Film
    films = relationship('Film', back_populates='language')

    def __repr__(self):
        return f"<Language(language_id={self.language_id}, name={self.name})>"

class Film(Base):
    """
    ORM model representing a film in the database.
    
    Relationships:
    - `actors`: Many-to-many relationship with the `Actor` model through the `film_actor` association table.
    - `reviews`: One-to-many relationship with the `Review` model.
    - `language`: Many-to-one relationship with the `Language` model.

    Attributes:
        film_id (int): Primary key, unique identifier for the film.
        title (str): The title of the film.
        description (str): A brief description of the film.
        language_id (int): Foreign key referencing the `Language` model.
    """
    __tablename__ = 'film'

    film_id = Column(Integer, primary_key=True, autoincrement=True)
    title = Column(String, nullable=False)
    description = Column(String)
    language_id = Column(Integer, ForeignKey('language.language_id'), nullable=False)

    # Many-to-many relationship with Actor
    actors = relationship('Actor', secondary=film_actor, back_populates='films')
    
    # One-to-many relationship with Review
    reviews = relationship('Review', back_populates='film')
    
    # Many-to-one relationship with Language
    language = relationship('Language', back_populates='films')

    def __repr__(self):
        return f"<Film(film_id={self.film_id}, title={self.title})>"

class Review(Base):
    """
    ORM model representing a review for a film in the database.
    
    Relationships:
    - `film`: Many-to-one relationship with the `Film` model.

    Attributes:
        review_id (int): Primary key, unique identifier for the review.
        content (str): The content of the review.
        rating (int): The rating of the film.
        film_id (int): Foreign key referencing the `Film` model.
    """
    __tablename__ = 'review'

    review_id = Column(Integer, primary_key=True, autoincrement=True)
    content = Column(String, nullable=False)
    rating = Column(Integer, nullable=False)
    film_id = Column(Integer, ForeignKey('film.film_id'), nullable=False)

    # Many-to-one relationship with Film
    film = relationship('Film', back_populates='reviews')

    def __repr__(self):
        return f"<Review(review_id={self.review_id}, rating={self.rating}, film_id={self.film_id})>"

# Create all tables in the database (if they don't exist)
Base.metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# Example: Add a new film with a language
# Assume the language ID for English is 1
english = session.query(Language).filter_by(name='English').first()

new_film = Film(title='Inception', description='A mind-bending thriller', language_id=english.language_id)
new_actor = Actor(first_name='Leonardo', last_name='DiCaprio')
new_film.actors.append(new_actor)

#new_review1 = Review(content='Amazing movie with stunning visuals!', rating=9, film=new_film)
#new_review2 = Review(content='Complex and intriguing storyline.', rating=8, film=new_film)

session.add(new_film)
session.commit()

# Selecting Results (Querying the database)
print("Films featuring Leonardo DiCaprio:")
leo = session.query(Actor).filter_by(first_name='Leonardo', last_name='DiCaprio').first()
for film in leo.films:
    print(f"- {film.title} ({film.description})")
    for review in film.reviews:
        print(f"  Review: {review.content} (Rating: {review.rating})")

# Updating a Row (Modify review rating)
review_to_update = session.query(Review).filter_by(content='Complex and intriguing storyline.').first()
review_to_update.rating = 10  # Change rating to 10
session.commit()

print("\nUpdated Reviews:")
for review in session.query(Review).all():
    print(f"- {review.content} (Rating: {review.rating})")

# Close the session after completing the queries
session.close()




Films featuring Leonardo DiCaprio:
- Inception (A mind-bending thriller)
  Review: Amazing movie with stunning visuals! (Rating: 9)
  Review: Complex and intriguing storyline. (Rating: 10)

Updated Reviews:
- Amazing movie with stunning visuals! (Rating: 9)
- Complex and intriguing storyline. (Rating: 10)
- Amazing movie with stunning visuals! (Rating: 9)
- Complex and intriguing storyline. (Rating: 8)
- Amazing movie with stunning visuals! (Rating: 9)
- Complex and intriguing storyline. (Rating: 8)


/var/folders/c1/8mhcv8nx2b9dswvv8thjqxjc0000gn/T/ipykernel_14973/300193988.py:17: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
